In [687]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumansharmadataworld/depression-surveydataset-for-analysis")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/sumansharmadataworld/depression-surveydataset-for-analysis/versions/1


In [688]:
import os
os.listdir(path)

['final_depression_dataset_1.csv']

In [689]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [690]:
df = pd.read_csv(f"{path}/final_depression_dataset_1.csv")
df.head()

,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Pooja,Female,37,Ghaziabad,Working Professional,Teacher,NaN,2.0,NaN,NaN,4.0,7-8 hours,Moderate,MA,No,6,2,No,No
1,Reyansh,Male,60,Kalyan,Working Professional,Financial Analyst,NaN,4.0,NaN,NaN,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,Yes,No
2,Manvi,Female,42,Bhopal,Working Professional,Teacher,NaN,2.0,NaN,NaN,3.0,5-6 hours,Moderate,M.Com,No,0,2,No,No
3,Isha,Female,44,Thane,Working Professional,Teacher,NaN,3.0,NaN,NaN,5.0,7-8 hours,Healthy,MD,Yes,1,2,Yes,No
4,Aarav,Male,48,Indore,Working Professional,UX/UI Designer,NaN,4.0,NaN,NaN,3.0,7-8 hours,Moderate,BE,Yes,6,5,Yes,No


In [691]:
df.shape

(2556, 19)

In [692]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Name                                   2556 non-null   object 
 1   Gender                                 2556 non-null   object 
 2   Age                                    2556 non-null   int64  
 3   City                                   2556 non-null   object 
 4   Working Professional or Student        2556 non-null   object 
 5   Profession                             1883 non-null   object 
 6   Academic Pressure                      502 non-null    float64
 7   Work Pressure                          2054 non-null   float64
 8   CGPA                                   502 non-null    float64
 9   Study Satisfaction                     502 non-null    float64
 10  Job Satisfaction                       2054 non-null   float64
 11  Slee

In [693]:
df.drop(['Name', 'Academic Pressure', 'CGPA', 'Study Satisfaction', 'City'], axis=1, inplace=True)

In [694]:
df['Working Professional or Student'].unique()

array(['Working Professional', 'Student'], dtype=object)

In [695]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [696]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Gender                                 1883 non-null   object 
 1   Age                                    1883 non-null   int64  
 2   Working Professional or Student        1883 non-null   object 
 3   Profession                             1883 non-null   object 
 4   Work Pressure                          1883 non-null   float64
 5   Job Satisfaction                       1883 non-null   float64
 6   Sleep Duration                         1883 non-null   object 
 7   Dietary Habits                         1883 non-null   object 
 8   Degree                                 1883 non-null   object 
 9   Have you ever had suicidal thoughts ?  1883 non-null   object 
 10  Work/Study Hours                       1883 non-null   int64  
 11  Fina

In [697]:
df.head()

,Gender,Age,Working Professional or Student,Profession,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Female,37,Working Professional,Teacher,2.0,4.0,7-8 hours,Moderate,MA,No,6,2,No,No
1,Male,60,Working Professional,Financial Analyst,4.0,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,Yes,No
2,Female,42,Working Professional,Teacher,2.0,3.0,5-6 hours,Moderate,M.Com,No,0,2,No,No
3,Female,44,Working Professional,Teacher,3.0,5.0,7-8 hours,Healthy,MD,Yes,1,2,Yes,No
4,Male,48,Working Professional,UX/UI Designer,4.0,3.0,7-8 hours,Moderate,BE,Yes,6,5,Yes,No


In [698]:
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [699]:
df['Gender'] = df['Gender'].map({'Female': 0, 'Male': 1})

In [700]:
df.head()

,Gender,Age,Working Professional or Student,Profession,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,37,Working Professional,Teacher,2.0,4.0,7-8 hours,Moderate,MA,No,6,2,No,No
1,1,60,Working Professional,Financial Analyst,4.0,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,Yes,No
2,0,42,Working Professional,Teacher,2.0,3.0,5-6 hours,Moderate,M.Com,No,0,2,No,No
3,0,44,Working Professional,Teacher,3.0,5.0,7-8 hours,Healthy,MD,Yes,1,2,Yes,No
4,1,48,Working Professional,UX/UI Designer,4.0,3.0,7-8 hours,Moderate,BE,Yes,6,5,Yes,No


In [701]:
df['Working Professional or Student'].unique()

array(['Working Professional'], dtype=object)

In [702]:
df.drop('Working Professional or Student', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

In [703]:
df.head()

,Gender,Age,Profession,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,37,Teacher,2.0,4.0,7-8 hours,Moderate,MA,No,6,2,No,No
1,1,60,Financial Analyst,4.0,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,Yes,No
2,0,42,Teacher,2.0,3.0,5-6 hours,Moderate,M.Com,No,0,2,No,No
3,0,44,Teacher,3.0,5.0,7-8 hours,Healthy,MD,Yes,1,2,Yes,No
4,1,48,UX/UI Designer,4.0,3.0,7-8 hours,Moderate,BE,Yes,6,5,Yes,No


In [704]:
enc = OneHotEncoder(categories='auto')
profession = df['Profession'].values.reshape(-1, 1)
enc.fit(profession)
new_features =  enc.get_feature_names_out()
print(new_features)
new_profession = pd.DataFrame(enc.transform(profession).toarray())

['x0_Accountant' 'x0_Architect' 'x0_Business Analyst' 'x0_Chef'
 'x0_Chemist' 'x0_Civil Engineer' 'x0_Consultant' 'x0_Content Writer'
 'x0_Customer Support' 'x0_Data Scientist' 'x0_Digital Marketer'
 'x0_Doctor' 'x0_Educational Consultant' 'x0_Electrician'
 'x0_Entrepreneur' 'x0_Finanancial Analyst' 'x0_Financial Analyst'
 'x0_Graphic Designer' 'x0_HR Manager' 'x0_Investment Banker' 'x0_Judge'
 'x0_Lawyer' 'x0_Manager' 'x0_Marketing Manager' 'x0_Mechanical Engineer'
 'x0_Pharmacist' 'x0_Pilot' 'x0_Plumber' 'x0_Research Analyst'
 'x0_Researcher' 'x0_Sales Executive' 'x0_Software Engineer' 'x0_Teacher'
 'x0_Travel Consultant' 'x0_UX/UI Designer']


In [705]:
new_profession.columns = ['Accountant', 'Architect', 'Business Analyst', 'Chef', 'Chemist', 'Civil Engineer', 'Consultant', 'Content Writer', 'Customer Support', 'Data Scientist', 'Digital Marketer', 'Doctor', 'Educational Consultant', 'Electrician', 'Entrepreneur', 'Finanancial Analyst', 'Financial Analyst', 'Graphic Designer', 'HR Manager', 'Investment Banker', 'Judge', 'Lawyer', 'Manager', 'Marketing Manager', 'Mechanical Engineer', 'Pharmacist', 'Pilot', 'Plumber', 'Research Analyst', 'Researcher', 'Sales Executive', 'Software Engineer', 'Teacher', 'Travel Consultant', 'UX/UI Designer']

In [706]:
df = pd.concat([df, new_profession], axis=1)
df.drop('Profession', axis=1, inplace=True)

In [707]:
df.head()

,Gender,Age,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,...,Pharmacist,Pilot,Plumber,Research Analyst,Researcher,Sales Executive,Software Engineer,Teacher,Travel Consultant,UX/UI Designer
0,0,37,2.0,4.0,7-8 hours,Moderate,MA,No,6,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,60,4.0,3.0,5-6 hours,Unhealthy,B.Com,Yes,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,42,2.0,3.0,5-6 hours,Moderate,M.Com,No,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,44,3.0,5.0,7-8 hours,Healthy,MD,Yes,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,48,4.0,3.0,7-8 hours,Moderate,BE,Yes,6,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [708]:
df['Sleep Duration'].unique()

array(['7-8 hours', '5-6 hours', 'More than 8 hours', 'Less than 5 hours'],
      dtype=object)

In [709]:
df['Sleep Duration'] = df['Sleep Duration'].map({'Less than 5 hours': 0, '5-6 hours': 1, '7-8 hours': 2, 'More than 8 hours': 3})

In [710]:
df.head()

,Gender,Age,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,...,Pharmacist,Pilot,Plumber,Research Analyst,Researcher,Sales Executive,Software Engineer,Teacher,Travel Consultant,UX/UI Designer
0,0,37,2.0,4.0,2,Moderate,MA,No,6,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,60,4.0,3.0,1,Unhealthy,B.Com,Yes,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,42,2.0,3.0,1,Moderate,M.Com,No,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,44,3.0,5.0,2,Healthy,MD,Yes,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,48,4.0,3.0,2,Moderate,BE,Yes,6,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [711]:
df['Dietary Habits'] = df['Dietary Habits'].map({'Unhealthy': 0, 'Moderate': 1, 'Healthy': 2})

In [712]:
df['Degree'].unique()

array(['MA', 'B.Com', 'M.Com', 'MD', 'BE', 'MCA', 'BA', 'LLM', 'BCA',
       'B.Ed', 'M.Tech', 'LLB', 'B.Arch', 'ME', 'MBA', 'M.Pharm', 'PhD',
       'BSc', 'MBBS', 'MHM', 'BBA', 'BHM', 'MSc', 'B.Tech', 'M.Ed',
       'B.Pharm'], dtype=object)

In [713]:
enc2 = OneHotEncoder(categories='auto')
degree = df['Degree'].values.reshape(-1, 1)
enc2.fit(degree)
new_features = enc2.get_feature_names_out()
print(new_features)
new_degree = pd.DataFrame(enc2.transform(degree).toarray())

['x0_B.Arch' 'x0_B.Com' 'x0_B.Ed' 'x0_B.Pharm' 'x0_B.Tech' 'x0_BA'
 'x0_BBA' 'x0_BCA' 'x0_BE' 'x0_BHM' 'x0_BSc' 'x0_LLB' 'x0_LLM' 'x0_M.Com'
 'x0_M.Ed' 'x0_M.Pharm' 'x0_M.Tech' 'x0_MA' 'x0_MBA' 'x0_MBBS' 'x0_MCA'
 'x0_MD' 'x0_ME' 'x0_MHM' 'x0_MSc' 'x0_PhD']


In [714]:
new_degree.columns = ['B.Arch', 'B.Com', 'B.Ed', 'B.Pharm', 'B.Tech', 'BA', 'BBA', 'BCA', 'BE', 'BHM', 'BSc', 'LLB', 'LLM', 'M.Com', 'M.Ed', 'M.Pharm', 'M.Tech', 'MA', 'MBA', 'MBBS', 'MCA', 'MD', 'ME', 'MHM', 'MSc', 'PhD']

In [715]:
df = pd.concat([df, new_degree], axis=1)
df.drop('Degree', axis=1, inplace=True)

In [716]:
df.head()

,Gender,Age,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,...,M.Tech,MA,MBA,MBBS,MCA,MD,ME,MHM,MSc,PhD
0,0,37,2.0,4.0,2,1,No,6,2,No,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,60,4.0,3.0,1,0,Yes,0,4,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,42,2.0,3.0,1,1,No,0,2,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,44,3.0,5.0,2,2,Yes,1,2,Yes,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,48,4.0,3.0,2,1,Yes,6,5,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [717]:
df['Have you ever had suicidal thoughts ?'] = df['Have you ever had suicidal thoughts ?'].map({'No': 0, 'Yes': 1})
df['Family History of Mental Illness'] = df['Family History of Mental Illness'].map({'No': 0, 'Yes': 1})
df['Depression'] = df['Depression'].map({'No': 0, 'Yes': 1})

In [718]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1883 entries, 0 to 1882
Data columns (total 72 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Gender                                 1883 non-null   int64  
 1   Age                                    1883 non-null   int64  
 2   Work Pressure                          1883 non-null   float64
 3   Job Satisfaction                       1883 non-null   float64
 4   Sleep Duration                         1883 non-null   int64  
 5   Dietary Habits                         1883 non-null   int64  
 6   Have you ever had suicidal thoughts ?  1883 non-null   int64  
 7   Work/Study Hours                       1883 non-null   int64  
 8   Financial Stress                       1883 non-null   int64  
 9   Family History of Mental Illness       1883 non-null   int64  
 10  Depression                             1883 non-null   int64  
 11  Acco

In [719]:
y = df['Depression'].values
X = df.drop('Depression', axis=1)

In [720]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(1883, 71)
(1883, 11)


In [721]:
# build mask
gender_0_mask = df['Gender'] == 0
gender_1_mask = df['Gender'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  909
Male:  974


In [722]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [723]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.000000  1.000000  0.000000  1.000000       0      88   
1      2  Female  0.666667  1.000000  0.000000  0.333333       2      88   
2      3  Female  0.750000  0.987342  0.012658  0.250000       9      78   
3      4  Female  0.857143  1.000000  0.000000  0.142857       6      84   
4      5  Female  0.571429  1.000000  0.000000  0.428571       4      84   
5      6  Female  0.500000  1.000000  0.000000  0.500000       4      83   
6      7  Female  0.642857  1.000000  0.000000  0.357143       9      77   
7      8  Female  0.818182  1.000000  0.000000  0.181818       9      80   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [724]:
results_df = pd.concat(results_list, ignore_index=True)

In [725]:
result_path = './results/k140_result.xlsx'
results_df.to_excel(result_path, index=False)

In [726]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.000000,1.000000,0.000000,1.000000,0,88,0,3,...,0,1,0.000000,1.000000,0.000000,1.000000,0,88,0,3
1,2,Female,0.666667,1.000000,0.000000,0.333333,2,88,0,1,...,0,1,0.666667,1.000000,0.000000,0.333333,2,88,0,1
2,3,Female,0.750000,0.987342,0.012658,0.250000,9,78,1,3,...,1,0,0.666667,0.987342,0.012658,0.333333,8,78,1,4
3,4,Female,0.857143,1.000000,0.000000,0.142857,6,84,0,1,...,0,0,0.857143,1.000000,0.000000,0.142857,6,84,0,1
4,5,Female,0.571429,1.000000,0.000000,0.428571,4,84,0,3,...,0,1,0.428571,1.000000,0.000000,0.571429,3,84,0,4


In [727]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: TtestResult(statistic=np.float64(1.3432765653428531), pvalue=np.float64(0.19587453855809672), df=np.float64(18.0))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(50.5), pvalue=np.float64(1.0))
SVM - FN/FP: TtestResult(statistic=np.float64(-1.6805557957512227), pvalue=np.float64(0.1101199097580026), df=np.float64(18.0))
DT -TPR: TtestResult(statistic=np.float64(0.5359802047223033), pvalue=np.float64(0.59853379569896), df=np.float64(18.0))
DT - FPR: TtestResult(statistic=np.float64(-1.3014403241354995), pvalue=np.float64(0.20951965647449858), df=np.float64(18.0))
DT - FN/FP: MannwhitneyuResult(statistic=np.float64(49.0), pvalue=np.float64(0.969815931818382))
RF -TPR: TtestResult(statistic=np.float64(0.5535311293239112), pvalue=np.float64(0.5867104133696756), df=np.float64(18.0))
RF - FPR: MannwhitneyuResult(statistic=np.float64(52.0), pvalue=np.float64(0.8710527848685038))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(38.5), pvalue=np.float64(0.39720953922612223))
LR 